Here is JINT code + adaptation to the real world.
# Contents


[**Racionale**](#proveThePoint)

**Hardware**
- [Decide flight platform and ros version](#flightPlatform)
- [Hardware Architecture](#hardwareArchitecture)

**Method**
- [Docker](#dockerFile)
- [Gazebo simulation](#gazebo)
- [Architecture](#architecture)
- Implementation challenges

[**Experiment**](#experiment)
- [Launching instructions](#LaunchingInstructions)

**Result analysis - Were there any improvements?**
The data to be used will be the flights in the testbed.

- RAM memory implications
- Computation time of paths

**Bibliography**
- Journal of Field Robotics prev implementation
- JINT / ICUAS



[test link](Paper A.ipynb)

<a id='proveThePoint'></a>
# Racionale

This approach adds the abitility to inspect any building, without any prior knowledge (not even on google maps), in all space of the 3 dimensions (not just ground or pillars or wall).

The final tests must prove it works on the target scale (oilrigs). This will be done through scaling the bootleneck points (distances, memory, sensor resolution). [study here](Paper C - Large structures test.ipynb#scaling) and  [calculation sheet](https://docs.google.com/spreadsheets/d/1UFneS__9-L0viD8dhHrdOTUGh9j_4SgYT0UNm2cHwcY/edit?usp=sharing)

Obstacle avoidance online.



<a id='flightPlatform'></a>
# Flight platform & software requirements

- **Payload** Point cloud sensor + camera for later mapping
- **Autopilot** Pixhawk 1 (for vicon compatibility)
- **Onboard CPU/GPU** UPboard
- **Vicon interface** is ROS_VRPN_CLIENT, working with Pixhawk 1 (Fran has experience)
- [**Gazebo out of the box**](http://gazebosim.org/tutorials?tut=ros_wrapper_versions#Gazebo7.xseries) 
    - Kinetic works with Gazebo 8 and Xenial [here](https://github.com/CentroEPiaggio/iliad_wiki/wiki/1.-ROS-Kinetic-and-Gazebo8-setup) is an installation guide
    - [Docker with PxHawk + Gazebo + mavros](https://dev.px4.io/en/test_and_ci/docker.html)
- **Ros version** (see above)
- **Docker** Likely yes 

[Excel for calculations](http://localhost:8888/?token=87db5327c113e4e5779dc262cbc3c08597b230f34cb1927c
)

<a id='hardwareArchitecture'></a>
# Hardware Architecture

[PixHawk Offboard mode](https://dev.px4.io/en/ros/offboard_control.html)

<a id='dockerFile'></a>
# Docker file


Problems with the networking part of things. Especially the GUI interface. Gui will be used for development for visual check of path generated (Lazy Theta*) and understanding the patterns generated by automated exploration. 

On the other hand, the big contribution of docker would be to run simultaneously experiments with different parameters, if these need visual confirmation it's pointless. 

BIG QUESTIONS: 
- How do I automate the assertion of the validity of a run? 
- How do I make videos for dissemination? (this is actually easy, just run locally one that you want to tape)

###### Following [Network Setup tutorial](http://wiki.ros.org/ROS/NetworkSetup) in same container
This turn out to be harder than anticipated. Made a question in [ROS Answers](https://answers.ros.org/question/273281/unable-to-setup-docker-on-kinetic/). Progress:
- If I start roscore on one bash of the container, it is not found on another bash of the same container. Looking into environment variables --> it seems I need to do `export ROS_MASTER_URI=http://172.17.0.2:11311/'

Instructions so far, using garuda as the name of the container: 

1. Start the container 
```
sudo docker run --name garuda -it osrf/ros:kinetic-desktop-xenial
```
    
2. In the local machine find container's IP
```
docker inspect garuda | grep IP
```
    
3. Install minimum stuff and start roscore
```
apt-get update
apt-get install wget
wget http://packages.ros.org/ros.key -O - |  apt-key add -
apt-get update
apt-get install python-catkin-tools
export ROS_MASTER_URI=http://<ip>:11311/
roscore
```    
4. Go to a different terminal and start the listener
```
docker exec -it garuda bash
export ROS_MASTER_URI=http://<ip>:11311/
source /opt/ros/ROS_DISTRO/setup.bash
rosrun rospy_tutorials listener.py
```
5. Go to yet another different terminal and start the talker 
```
docker exec -it garuda bash
export ROS_MASTER_URI=http://<ip>:11311/
source /opt/ros/$ROS_DISTRO/setup.bash
rosrun rospy_tutorials talker.py
```

###### Setup communication as two networked ROS machines, one docker and the other normal desktop. Following [Network Setup tutorial](http://wiki.ros.org/ROS/NetworkSetup)
1. Start the container with [-P flag was used to automatically map any network port inside it to a random high port within an ephemeral port range on your Docker host](https://docs.docker.com/engine/userguide/networking/default_network/dockerlinks/)
```
sudo docker run --name garuda -P -it osrf/ros:kinetic-desktop-xenial
```

2. Same but after `export ROS_MASTER_URI` run `export ROS_IP=<container ip>`

3. Same but after `export ROS_MASTER_URI` run `export ROS_IP=<container ip>`

4. Same but in local machine. 
    - Roscore hears the message but issues the following warning `Couldn't find an AF_INET address for [mfaria-Workstation]`.
    - If additionally the ROS_IP is set to the local machine IP it does not work and no message appears on roscore. 

###### Docker-compose
From the PhD student that setup the farms at OSRF an [example to use docker with docker-compose](https://gist.github.com/ruffsl/4a24c26a1aa2cc733c64)

1. [Install docker compose](https://docs.docker.com/compose/install/#install-compose) 
```sudo curl -L https://github.com/docker/compose/releases/download/3.2.0/docker-compose-`uname -s`-`uname -m` -o /usr/local/bin/docker-compose
sudo chmod +x /usr/local/bin/docker-compose```
2. Create a file with the files on the [gist](https://gist.github.com/ruffsl/4a24c26a1aa2cc733c64) and go there
3. `docker-compose up`
The original example is in indigo. If this works will need to upgrade TODO
4. The containers can talk with each other perfectly and the host can ping all of them. Meaning the following instructions work:

```
ping master
ping talker
ping listener```

However listing topics does not:

```ROS_MASTER_URI=http://master:11311
rostopic list
ERROR: Unable to communicate with master!
```

This was done with composer version 1.16.0 instead of the intended 1.6.2. Will revert to correcto composer version and try again.

###### Possible options for setting up the network
- I did not understood why the previous fails, as the inner workings of AF_INET and hosts are still unclear. However the developer involved in creating Open Source Robotics Foundation's farms contibuted a way of [launching several containers with docker-composer](https://gist.github.com/ruffsl/4a24c26a1aa2cc733c64).
- [Q Change hosts file](https://answers.ros.org/question/254269/docker-ros-unable-to-communicate-with-master/)
- [Q Remove network isolation](https://answers.ros.org/question/250054/how-to-use-docker-on-a-host-together-with-non-docker-machines-ros/)
- [Q Use vm network ip in ROS_MASTER_URI instead of domain name, for VM](https://answers.ros.org/question/227934/roscore-on-virtual-machine/)
- Checkout the docs for [intel aero platform with docker](https://github.com/intel-aero/meta-intel-aero/wiki/08-Aero-Network-and-System-Administration#sysadmin-docker-ros)

###### Docker for px4 + gazebo

[Installation guide](https://dev.px4.io/en/test_and_ci/docker.html)
It is not a very good one.
So far I additionally had to do:
1. create a dedicated folder 
1. cd into it
1. `git clone https://github.com/PX4/Firmware.git`
1. only then start on the installation guide. The provided command was adapted into <pre><code>docker run -it --privileged -v \`pwd\`:\`pwd\`:rw -v /tmp/.X11-unix:/tmp/.X11-unix:ro -e DISPLAY=:0 -p 14556:14556/udp --name=px4_gazebo px4io/px4-dev-ros:2017-05-31 bash</code></pre>
1. Then leave to enter as root
<pre><code>exit
xhost +
docker exec -u 0 -it px4_gazebo bash</pre></code>
1. Go into the Firmware folder and continue the installation
<pre><code>make posix_sitl_default gazebo</pre></code>

Issue submitted to px4 [here](https://github.com/PX4/containers/issues/81)

Need to do `xhost +` every time, but after that gazebo works. As does everything else.

**Seemed final but had the mount permission issue for workspace**

Created Dockerfile with px4-dev-ros:2017-10-23 as base and px4 firmware tag v1.6.5 (on git)

Built with  `sudo docker build -t garuda_sitl .` while at _docker folder

To launch:
1. `xhost +`
2. `docker run -it --privileged -v /home/mfaria/ws_mavlink_grvcHal/:/home/user/ros_workspace:rw -v /tmp/.X11-unix:/tmp/.X11-unix:ro -e DISPLAY=:0 -p 14556:14556/udp --name=px4_gazebo garuda_sitl bash`
3. `exit`
4. `docker start px4_gazebo`
4. `docker exec -u 0 -it px4_gazebo bash`
5. Go into the Firmware folder and continue the installation `make posix_sitl_default gazebo`


###### Firmware by volume mount

- Removed clone from dockerfile
- Launching as `docker run -it --privileged \
    -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
    -e DISPLAY=:0 \
    --name=garuda_sitl px4io/px4-dev-ros:2017-12-08 /bin/bash`
- Launching as `sudo docker run -it --privileged     -v /home/mfaria/Flying_Octomap:/ros_ws     -v /tmp/.X11-unix:/tmp/.X11-unix:ro     -e DISPLAY=:0     --name=garuda_sitl px4io/px4-dev-ros:2017-12-08 /bin/bash`
- From docker px4io/px4-dev-ros tag 2017-12-08
    1. Clone Firmware folder
        1. Download git `git clone https://github.com/PX4/Firmware.git`
        1. Change to tag `git checkout v1.7.0-rc4`
    2. Build docker image
        1. `cd /home/mfaria/Flying_Octomap/src/_docker`
        1. Built with  `docker build -t garuda_sitl .`
    3. Launch docker
        1. `xhost +`
        1. Lauch docker `docker run -it --privileged \
        -v /home/mfaria/docker_px4/Firmware:/Firmware:rw \
        -v /home/mfaria/Flying_Octomap/src:/ros_ws/src:rw \
        -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
        -e DISPLAY=:0 \
        --name=garuda_sitl px4io/px4-dev-ros:2017-12-08 /bin/bash`
        1. exit
        1. `docker start garuda_sitl`
        1. `docker exec -u 0 -it garuda_sitl bash`
    1. Make ros workspace:
        1. `source /opt/ros/kinetic/setup.bash`
        2. `catkin_make`
    1. To launch gazebo: 
        1. `cd /Firmware`
        1. `make posix_sitl_default gazebo`
- From docker custom image garuda_sitl_v2 based on px4io/px4-dev-ros tag 2017-12-08
    1. Clone Firmware folder
        1. Download git `git clone https://github.com/PX4/Firmware.git`
        1. Change to tag `git checkout v1.7.0-rc4`
    2. Build docker image
        1. Built with  `docker build -t garuda_sitl_v2 .` while at _docker folder
    3. Launch docker
        1. `xhost +`
        1. Lauch docker `docker run -it --privileged \
        -v /home/mfaria/docker_px4/Firmware:/Firmware:rw \
        -v /home/mfaria/Flying_Octomap/src:/ros_ws/src:rw \
        -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
        -e DISPLAY=:0 \
        --name=garuda_sitl_v2 garuda_sitl_v2 /bin/bash`
        1. exit
        1. `docker start garuda_sitl_v2`
        1. `docker exec -u 0 -it garuda_sitl_v2 bash`
    1. Make ros workspace:
        1. `source /opt/ros/kinetic/setup.bash`
        1. `cd /ros_ws`
        2. `catkin init`
    1. Build project:
        1. `catkin build` (should take around 3 min)
    1. To launch gazebo: 
        1. `cd /Firmware`
        1. `make posix_sitl_default gazebo`

<a id='gazebo'></a>
# Gazebo and SITL and MavRos

##### Overlaying for launch
[Overlaying docs](http://wiki.ros.org/catkin/Tutorials/workspace_overlaying)
[Launching gazebo and pc4 separatly docs](https://dev.px4.io/en/simulation/ros_interface.html)

** Try 1 **
1. Docker
    1. Clone Firmware folder
        1. Download git `git clone https://github.com/PX4/Firmware.git`
        1. Change to tag `git checkout v1.7.0-rc4`
    2. Build docker image
        1. Built with  `docker build -t garuda_sitl_v2 .` while at _docker folder
    3. Launch docker
        1. `xhost +`
        1. Lauch docker `docker run -it --privileged \
        -v /home/mfaria/docker_px4/Firmware:/Firmware:rw \
        -v /home/mfaria/ws_mavlink_grvcHal/src:/ros_ws/src:rw \
        -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
        -e DISPLAY=:0 \
        --name=garuda_sitl_v2 garuda_sitl_v2 /bin/bash`
        1. exit
        1. `docker start garuda_sitl_v2`
        1. `docker exec -u 0 -it garuda_sitl_v2 bash`
1. Build with overlay
    1. Build base = px4. Keep terminal
        1. `cd /Firmware`
        1. `no_sim=1 make posix_sitl_default gazebo`
    1. Build overlay
        1. New terminal (check launch docker)
        1. (First time) Make ros workspace:
            1. `source /opt/ros/kinetic/setup.bash`
            1. `cd /ros_ws`
            2. `catkin init`
        1. `catkin build` (should take around 3 min)
        1. Launch gazebo w/ iris
            1. `roslaunch gazebo_ros empty_world.launch world_name:=/Firmware/Tools/sitl_gazebo/worlds/iris.world`
        
** Try 2 **
1. Docker
    1. Clone Firmware folder
        1. Download git `git clone https://github.com/PX4/Firmware.git`
        1. Change to tag `git checkout v1.7.0-rc4`
    2. Build docker image
        1. Built with  `docker build -t garuda_sitl_v2 .` while at _docker folder
    3. Launch docker
        1. `xhost +`
        1. `docker run -it --privileged \
        -v /home/mfaria/docker_px4/Firmware:/Firmware:rw \
        -v /home/mfaria/ws_mavlink_grvcHal/src:/ros_ws/src:rw \
        -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
        -e DISPLAY=:0 \
        --name=garuda_sitl_v2 garuda_sitl_v2 /bin/bash`
        1. exit
        1. `docker start garuda_sitl_v2`
        1. `docker exec -u 0 -it garuda_sitl_v2 bash`
1. Build ros workspace
    1. (First time) Make ros workspace:
        1. `source /opt/ros/kinetic/setup.bash`
        1. `cd /ros_ws`
        2. `catkin init`
    1. `catkin build` (should take around 3 min)
1. Build following [px4](https://dev.px4.io/en/simulation/ros_interface.html) instructions - keep terminal
    1. `cd /Firmware`
    1. `make posix_sitl_default gazebo`
    1. Kill process (this was just for the make part, they apperantly complicated the toolchain at px4)
    1. `source /ros_ws/devel/setup.bash`
    1. `source Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo`
    1. `roslaunch px4 posix_sitl.launch`
1. New terminal
    1. `xhost +`
    1. `docker exec -u 0 -it garuda_sitl_v2 bash`
    1. `cd /Firmware`
    1. `source /ros_ws/devel/setup.bash`
    1. `source Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo`
1. Launch mavros node `roslaunch mavros px4.launch fcu_url:="udp://:14540@127.0.0.1:14557"`
1. In new terminal, launch offboard node `rosrun architecture px4_mavros_offboard_node`
    


**EUROC setup but with gazebo:**
- Vicon
- Px4 interfacing with ROS [docs](https://dev.px4.io/en/simulation/ros_interface.html)
`roslaunch mavros px4.launch fcu_url:="udp://:14540@127.0.0.1:14557"`

Launch with GDB for headless (?) `make posix_sitl_default gazebo_iris_gdb` from end of [Gazebo + px4 docs](https://dev.px4.io/en/simulation/gazebo.html)

##### Launch file
- Model UAV + hokuyo sensor
- Node auto pilot relay


<a id='architecture'></a>
# Software Architecture (ROS)

[Architecture diagram](https://docs.google.com/drawings/d/1fRqLOSLiDlbrC7rG28KdJ4CaBSvM_ozNhlzRuhP2z98/edit?usp=sharing)

**Version 1** 

**Version 2** 

<a id='mavros'></a>
### Mavros

- [ROS wiki](http://wiki.ros.org/mavros) 
`roslaunch mavros px4.launch`
- [Off board example](https://dev.px4.io/en/ros/mavros_offboard.html)
- mavlink_adapter 

<a id='ObstacleAvoidance'></a>
### Obstacle Avoidance

- Check area with a bounding boz for security area
- Use cast rays with lower resolution. Might have some correlation issues with cell id
- Think of octomap as Voronoi Map. Always using a resolution that guarantees safety.

<a id='toImplement'></a>
### To Implement

- Lazy theta Star using unknwon cells
- Coverage calculation. Decide is an unknow cell is covered if it is unreachable. Decide the waypoint that needs to be visited to gather the information.

<a id='experiment'></a>
# Experiment

### Steps
1. Set volume of the testbead. No prior knowledge
2. Just press go
3. Feed data into pix4d
4. Show map

### What to measure
- Coverage
- Flight time needed in autonomous mode
- Flight time and planning time needed in Pix4D
- Flight time needed in old implementation
- Resolution
- Lazy Theta * iterations (old and new implementations)
- Lazy Theta * computation time for each waypoint (old and new implementation)
- RAM usage (old and new implementation)

<a id='LaunchingInstructions'></a>
### Launching Instructions

1. **Launch docker + gazebo + px4 sitl**
    1. `xhost +`
    1. Lauch docker `docker run -it --privileged \
    -v /home/mfaria/docker_px4/Firmware:/Firmware:rw \
    -v /home/mfaria/ws_mavlink_grvcHal/src:/ros_ws/src:rw \
    -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
    -e DISPLAY=:0 \
    --name=garuda_sitl_v2 garuda_sitl_v2 /bin/bash`
    1. exit
    1. `docker start garuda_sitl_v2`
    1. `docker exec -u 0 -it garuda_sitl_v2 bash`
1. Build ros workspace 
    ```
    source /opt/ros/kinetic/setup.bash
    cd /ros_ws
    catkin build
    ```
1. Build following [px4](https://dev.px4.io/en/simulation/ros_interface.html) instructions - keep terminal
    ```
    cd /Firmware
    source /ros_ws/devel/setup.bash
    source Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default
    export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware
    export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo
    roslaunch px4 posix_sitl.launch
    ```
1. New terminal
    1. `xhost +`
    1. `docker exec -u 0 -it garuda_sitl_v2 bash`
    1. Setup path
    ```
    cd /Firmware
    source /ros_ws/devel/setup.bash
    source Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default
    export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware
    export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo
    ```
1. Launch mavros node `roslaunch mavros px4.launch fcu_url:="udp://:14540@127.0.0.1:14557"`
1. In new terminal, launch offboard node `rosrun architecture px4_mavros_offboard_node`